In [36]:
# from Modules.LSH import semantic_query_lsh
# from Modules.LSH import LSH


# import numpy as np
# import os

In [37]:
# file_path = "./random_data.txt"
# read_data = np.loadtxt(file_path)
# plane_norms = LSH(read_data, 8)
# query=[read_data[0]]
# folder_name = "bucket_files"
# result = semantic_query_lsh(query, plane_norms,folder_name)

In [38]:
%load_ext autoreload
%autoreload 2


from utils import *
from Modules.LSH import*
from api import *
from evaluation import *
from worst_case_implementation import VecDBWorst


datafile_path="../DataBase/random_data_10000.txt"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
data_file='../DataBase/data.bin'
Level_1_path='../DataBase/Level1'
Level_2_path='../DataBase/Level2'
Level_3_path='../DataBase/Level3'

Level_1_nbits=8
Level_2_nbits=3
Level_3_nbits=3

data_api = DataApi(data_file)
data_api.generate_data_file(10000)

Inserted 10000 records. Total records inserted: 10000
Insertion complete.


In [40]:

# Test LSH_index
# Read Data From File
read_data = data_api.get_first_k_records(10000)


# Layer(1)
level_1_in=read_data
# TODO: Save Planes to be used in query Search
level_1_planes=LSH_index(data=level_1_in, nbits=Level_1_nbits,index_path=Level_1_path)

In [41]:
# Layer(2)
# On Each Bucket Apply LSH

# List all files in the directory
files = os.listdir(Level_1_path)

# TODO: Save Planes to be used in query Search
level_2_planes={}

# Loop over the files
for file_name in files:
    file_path = os.path.join(Level_1_path, file_name)
    
    if os.path.isfile(file_path):
        # Read Data
        read_data_2 = np.loadtxt(file_path,dtype=int,ndmin=1)

        level_2_in=data_api.get_multiple_records_by_ids(read_data_2)
        # level_2_in = array_to_dictionary(values=vectors,keys=np.hstack(read_data_2))

        # # Apply LSH on this Bucket
        level_2_planes[file_name[:-4]]=LSH_index(data=level_2_in.values(), nbits=Level_2_nbits,index_path=Level_2_path+'/' + file_name[:-4])



In [42]:
# Layer(3)
# On Each Bucket Apply LSH

# List all files in the directory
folders = os.listdir(Level_2_path)

# TODO: Save Planes to be used in query Search
level_3_planes={}
# Loop over the folders
for folder_name in folders:
    folder_path = os.path.join(Level_2_path, folder_name)
    files = os.listdir(folder_path)
    # Loop over the files
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.isfile(file_path):
            # Read Data
            read_data_3 = np.loadtxt(file_path,dtype=int,ndmin=1)

            level_3_in=data_api.get_multiple_records_by_ids(read_data_3)

            # # Apply LSH on this Bucket
            level_3_planes[file_name[:-4]]=LSH_index(data=level_3_in.values(), nbits=Level_3_nbits,index_path=Level_3_path+'/'+folder_name+'/' + file_name[:-4])


In [43]:
# Query
query=data_api.get_record_by_id(5)[5]['embed']
# Layer (1)
bucket_1,index_result_1 = semantic_query_lsh(query=query,plane_norms=level_1_planes,index_path=Level_1_path)
print("level(1)",bucket_1)
print("level(1)",len(index_result_1))

# Layer(2)
bucket_2,index_result_2 = semantic_query_lsh(query=query,plane_norms=level_2_planes[bucket_1],index_path=Level_2_path+"/"+bucket_1)
print("level(2)",bucket_2)
print("level(2)",len(index_result_2))

# Layer(3)
bucket_3,index_result_3 = semantic_query_lsh(query=query,plane_norms=level_3_planes[bucket_2],index_path=Level_3_path+"/"+bucket_1+'/'+bucket_2)
print("level(3)",bucket_3)
print("level(3)",len(index_result_3))



count = sum(element in index_result_2 for element in index_result_3)
print(count)



level(1) 10001100
level(1) 3876
level(2) 110
level(2) 1734
level(3) 110
level(3) 17
17


In [44]:
np_rows = np.array([record['embed'] for record in read_data if 'embed' in record])

res=run_queries([4, 966, 536, 1088, 5073, 5549], np_rows, top_k=1, num_runs=1,query=np.array([query]))
print(eval(res))

Ours: [4, 966, 536, 1088, 5073, 5549]
His: [4, 9297, 1082, 1801, 7605, 5727, 5628, 8357, 5039, 3457, 5540, 8624, 8720, 6758, 9030, 8866, 3532, 266, 4661, 6727, 6891, 7066, 2488, 3503, 7873, 4068, 8230, 860, 7916, 3314, 7382, 5501, 9241, 3839, 4681, 9574, 946, 7240, 4037, 9775, 1871, 6720, 4004, 8164, 146, 8569, 3280, 2323, 6479, 7483, 1461, 1001, 9983, 7913, 5129, 2910, 7804, 8033, 6250, 7009, 1898, 9892, 4332, 3686, 5249, 2492, 3890, 6652, 8087, 6246, 9284, 9400, 3996, 5696, 5263, 2512, 8925, 5867, 7326, 3605, 6195, 1847, 6128, 1608, 3479, 9398, 2996, 334, 9303, 7161, 3587, 9194, 8635, 5937, 7404, 2615, 4320, 7379, 8583, 9573, 4820, 602, 2873, 1126, 3698, 3267, 6968, 6676, 2695, 8119, 1190, 3871, 5171, 3770, 4595, 2555, 8191, 8239, 3480, 7246, 1412, 3350, 3276, 1938, 1863, 2026, 1469, 4722, 2096, 7845, 7851, 5137, 5750, 5127, 4867, 1855, 2651, 1429, 7700, 641, 8270, 3105, 6409, 6920, 4313, 27, 4295, 9627, 57, 6378, 2472, 7350, 2876, 3484, 7572, 3830, 4385, 4358, 596, 9278, 1948, 1432,

In [45]:
db = VecDBWorst()
# records_np = np.random.random((10000, 70))
records_np = np.array([record['embed'] for record in read_data if 'embed' in record])

# records_dict = [{"id": i, "embed": list(row)} for i, row in enumerate(records_np)]
records_dict=read_data

# _len = len(records_np)
db.insert_records(records_dict)
db_ids=db.retrive(query, top_k=1)
res = run_queries(db_ids, records_np, 1, 1,np.array([query]))
print(eval(res))

Ours: [5]
His: [4, 9297, 1082, 1801, 7605, 5727, 5628, 8357, 5039, 3457, 5540, 8624, 8720, 6758, 9030, 8866, 3532, 266, 4661, 6727, 6891, 7066, 2488, 3503, 7873, 4068, 8230, 860, 7916, 3314, 7382, 5501, 9241, 3839, 4681, 9574, 946, 7240, 4037, 9775, 1871, 6720, 4004, 8164, 146, 8569, 3280, 2323, 6479, 7483, 1461, 1001, 9983, 7913, 5129, 2910, 7804, 8033, 6250, 7009, 1898, 9892, 4332, 3686, 5249, 2492, 3890, 6652, 8087, 6246, 9284, 9400, 3996, 5696, 5263, 2512, 8925, 5867, 7326, 3605, 6195, 1847, 6128, 1608, 3479, 9398, 2996, 334, 9303, 7161, 3587, 9194, 8635, 5937, 7404, 2615, 4320, 7379, 8583, 9573, 4820, 602, 2873, 1126, 3698, 3267, 6968, 6676, 2695, 8119, 1190, 3871, 5171, 3770, 4595, 2555, 8191, 8239, 3480, 7246, 1412, 3350, 3276, 1938, 1863, 2026, 1469, 4722, 2096, 7845, 7851, 5137, 5750, 5127, 4867, 1855, 2651, 1429, 7700, 641, 8270, 3105, 6409, 6920, 4313, 27, 4295, 9627, 57, 6378, 2472, 7350, 2876, 3484, 7572, 3830, 4385, 4358, 596, 9278, 1948, 1432, 2380, 9680, 4514, 3337, 331